# Import CSV Server aus dem Flatfile aufbauen
## TM1py Best Practices
Siehe TM1py Samples und im Speziellen [hier](https://github.com/cubewise-code/tm1py-samples/blob/master/Samples/samples_setup.py)
## Changelog
### 2024-02-16: 
neuen TM1 Server "csvimport" konfiguriert
### 2024-02-20:
generische Dimensionen Months, Years, etc
### 2024-02-22:
Produkte und Locations aus CSV
Cube Sales erstellt und aus CSV beladen

###ToDo:
direkt den DataFrame laden


In [1]:
# Flatfile schaut gut, das importieren wir in einen TM1 Cube
# 1) Verbindung zum Server prüfen
import pandas as pd
import configparser
from TM1py.Services import TM1Service
from TM1py.Objects import Cube, Dimension, Element, ElementAttribute, Hierarchy

config = configparser.ConfigParser()
try:
    # storing the credentials in a file is not recommended for purposes other than testing.
    # it's better to setup CAM with SSO or use keyring to store credentials in the windows credential manager. Sample:
    # Samples/credentials_best_practice.py
    config.read(r'./import/config.ini')

    with TM1Service(**config['csvimport']) as tm1:
        server_name = tm1.server.get_server_name()
        print("Connection to TM1 established!! your Servername is: {}".format(server_name))

except Exception as e:
    print("\nERROR:")
    print("\t" + str(e))

Connection to TM1 established!! your Servername is: csvimport


In [2]:
# push data to TM1
with TM1Service(**config['csvimport']) as tm1:
    # ============================
    # create dimension Months
    dim_month = 'Months'
    hier_month = dim_month
    elements = [Element(str(month), 'Numeric') for month in range(1, 13)]
    hierarchy = Hierarchy(dim_month, hier_month, elements)
    dimension = Dimension(dim_month, [hierarchy])
    # if not tm1.dimensions.exists(dimension.name):
        # tm1.dimensions.create(dimension)
    tm1.dimensions.update_or_create(dimension)

    # create dimension Years
    dim_year = 'Years'
    hier_year = dim_year
    elements = [Element(str(year), 'Numeric') for year in range(2000, 2030, 1)]
    hierarchy = Hierarchy(dim_year, hier_year, elements)
    dimension = Dimension(dim_year, [hierarchy])
    tm1.dimensions.update_or_create(dimension)

    # create dimension Measures
    dim_kpi = 'Measures'
    hier_kpi = dim_kpi
    measures = ('Quantity', 'Price')
    elements = [Element(kpi, 'Numeric') for kpi in measures]
    hierarchy = Hierarchy(dim_kpi, hier_kpi, elements)
    dimension = Dimension(dim_kpi, [hierarchy])
    tm1.dimensions.update_or_create(dimension)

## Schritt 2 - weitere Dimensionen auf Basis der CSV aufbauen
- Produkte aus den Bewegungsdaten aufbauen; erstmal nicht berücksichtigen, dass Produkte umbenannt werden
- CSV lesen und die Spalten unique für die Dimensionselemente einlesen

### CSV laden

In [3]:
df = pd.read_csv('./import/sales_data_sample.csv', encoding = 'latin1')
df.info()
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2823 non-null   int64  
 1   QUANTITYORDERED   2823 non-null   int64  
 2   PRICEEACH         2823 non-null   float64
 3   ORDERLINENUMBER   2823 non-null   int64  
 4   SALES             2823 non-null   float64
 5   ORDERDATE         2823 non-null   object 
 6   STATUS            2823 non-null   object 
 7   QTR_ID            2823 non-null   int64  
 8   MONTH_ID          2823 non-null   int64  
 9   YEAR_ID           2823 non-null   int64  
 10  PRODUCTLINE       2823 non-null   object 
 11  MSRP              2823 non-null   int64  
 12  PRODUCTCODE       2823 non-null   object 
 13  CUSTOMERNAME      2823 non-null   object 
 14  PHONE             2823 non-null   object 
 15  ADDRESSLINE1      2823 non-null   object 
 16  ADDRESSLINE2      302 non-null    object 


In [4]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2003-02-24,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,2003-05-07,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,2003-07-01,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,2003-08-25,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,2003-10-10,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [5]:
df['PRODUCTLINE'].value_counts()


PRODUCTLINE
Classic Cars        967
Vintage Cars        607
Motorcycles         331
Planes              306
Trucks and Buses    301
Ships               234
Trains               77
Name: count, dtype: int64

In [6]:
df.describe()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,MSRP
count,2823.000000,2823.000000,2823.000000,2823.000000,2823.000000,2823,2823.000000,2823.000000,2823.00000,2823.000000
mean,10258.725115,35.092809,83.658544,6.466171,3553.889072,2004-05-11 00:16:49.989373056,2.717676,7.092455,2003.81509,100.715551
min,10100.000000,6.000000,26.880000,1.000000,482.130000,2003-01-06 00:00:00,1.000000,1.000000,2003.00000,33.000000
25%,10180.000000,27.000000,68.860000,3.000000,2203.430000,2003-11-06 12:00:00,2.000000,4.000000,2003.00000,68.000000
50%,10262.000000,35.000000,95.700000,6.000000,3184.800000,2004-06-15 00:00:00,3.000000,8.000000,2004.00000,99.000000
75%,10333.500000,43.000000,100.000000,9.000000,4508.000000,2004-11-17 12:00:00,4.000000,11.000000,2004.00000,124.000000
max,10425.000000,97.000000,100.000000,18.000000,14082.800000,2005-05-31 00:00:00,4.000000,12.000000,2005.00000,214.000000
std,92.085478,9.741443,20.174277,4.225841,1841.865106,NaN,1.203878,3.656633,0.69967,40.187912


### Productline C-Elemente mit aufbauen
```
edges = {('Europe', 'CH'): 1,
        ('Europe', 'UK'): 1,
        ('Europe', 'BE'): 1}
```

In [7]:
locations = df[['COUNTRY', 'CITY']].drop_duplicates()

# Zirkelbezug, weil Singapore COUNTRY und CITY 
# Delete rows where the 'COUNTRY' is 'Singapore' or the 'CITY' is 'Singapore'
indexSingapore = locations[(locations['COUNTRY'] == 'Singapore') | (locations['CITY'] == 'Singapore')].index
locations.drop(indexSingapore, inplace=True)

dim_name = 'Locations'

# Connection to TM1. Needs IP, Port, Credentials, and SSL
with TM1Service(**config['csvimport']) as tm1:

    elements = []
    # create elements objects
    for city in locations['CITY'].unique():
        elements.append(Element(name = city, element_type = 'Numeric'))

    for country in locations['COUNTRY'].unique():
        elements.append(Element(name = country, element_type = 'Consolidated'))

    # create edge object
    # itertuple über DF und dann weight als Value für Dict
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.itertuples.html
    edges = {}
    for elpar in list(locations.itertuples(index=False, name=None)):
        edges[elpar] = 1

    # create the element_attributes
    element_attributes = [ElementAttribute(name='Name Long', attribute_type='Alias'),
                          ElementAttribute(name='Name Short', attribute_type='Alias'),
                          ElementAttribute(name='Currency', attribute_type='String')]

    # create hierarchy object
    hierarchy = Hierarchy(name=dim_name, dimension_name=dim_name, elements=elements, element_attributes=element_attributes,
                          edges=edges)

    # create dimension object
    d = Dimension(name=dim_name, hierarchies=[hierarchy])

    # create dimension in TM1 !
    tm1.dimensions.update_or_create(d)

In [8]:
products = df[['PRODUCTLINE', 'PRODUCTCODE']].drop_duplicates()

dim_name = 'Products'

# Connection to TM1. Needs IP, Port, Credentials, and SSL
with TM1Service(**config['csvimport']) as tm1:

    elements = []
    # create elements objects
    for code in products['PRODUCTCODE'].unique():
        elements.append(Element(name = code, element_type = 'Numeric'))

    for line in products['PRODUCTLINE'].unique():
        elements.append(Element(name = line, element_type = 'Consolidated'))

    # create edge object
    # itertuple über DF und dann weight als Value für Dict
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.itertuples.html
    edges = {}
    for elpar in list(products.itertuples(index=False, name=None)):
        edges[elpar] = 1

    # create the element_attributes
    element_attributes = [ElementAttribute(name='Name Long', attribute_type='Alias'),
                          ElementAttribute(name='Name Short', attribute_type='Alias'),
                          ElementAttribute(name='Currency', attribute_type='String')]

    # create hierarchy object
    hierarchy = Hierarchy(name=dim_name, dimension_name=dim_name, elements=elements, element_attributes=element_attributes,
                          edges=edges)

    # create dimension object
    d = Dimension(name=dim_name, hierarchies=[hierarchy])

    # create dimension in TM1 !
    tm1.dimensions.update_or_create(d)

## Cube erstellen & Daten laden
- Rule für Menge * Preis später hinzufügen

In [9]:
with TM1Service(**config['csvimport']) as tm1:
    cube = Cube(name='Sales', dimensions=['Years', 'Months', 'Locations', 'Products', 'Measures'], rules='')
    tm1.cubes.update_or_create(cube)

In [10]:
cube_name = 'Sales'
data = df[['YEAR_ID', 'MONTH_ID', 'CITY', 'PRODUCTCODE','QUANTITYORDERED','PRICEEACH']]
#data.insert(4, 'MEASURE', 'Quantity')

print(data.head())
with TM1Service(**config['csvimport']) as tm1:
    # tm1.cubes.cells.write_dataframe(cube_name, data)

    # cellset to store the new data
    cellset = {}
    # Populate cellset with coordinates and value pairs
    for index, row in data.iterrows():
        if not row['CITY'] == 'Singapore':
            cellset[(row['YEAR_ID'], row['MONTH_ID'], row['CITY'], row['PRODUCTCODE'], 'Quantity')] = row['QUANTITYORDERED']
            cellset[(row['YEAR_ID'], row['MONTH_ID'], row['CITY'], row['PRODUCTCODE'], 'Price')] = row['PRICEEACH']

    # send the cellset to TM1
    tm1.cubes.cells.write_values(cube_name, cellset)

   YEAR_ID  MONTH_ID           CITY PRODUCTCODE  QUANTITYORDERED  PRICEEACH
0     2003         2            NYC    S10_1678               30      95.70
1     2003         5          Reims    S10_1678               34      81.35
2     2003         7          Paris    S10_1678               41      94.74
3     2003         8       Pasadena    S10_1678               45      83.26
4     2003        10  San Francisco    S10_1678               49     100.00
